# MOFA+: training a model in Python
Author: Ricard Argelaguet.    
Affiliation: European Bioinformatics Institute, Cambridge, UK.  
Date: 13/11/2019  



This notebook contains a detailed tutorial on how to train MOFA using Python. If you arrived here, that means that you followed the Python path. 

A concise template script to run the code below can be found [here](XXX)



## 1) Load libraries

In [15]:
from mofapy2.run.entry_point import entry_point
import pandas as pd
import numpy as np

# initialise the entry point
ent = entry_point()



           |￣￣￣￣￣￣￣|
           |             |
           |    MOFA+    |
           |             |
           | ＿＿＿＿＿＿_|
           (\__/) ||
           (•ㅅ•)  ||
           / 　 づ
        


## 2) Load data

To create a MOFA+ object you need to specify four dimensions: samples (cells), features, view(s) and group(s). MOFA objects can be created from a wide range of input formats:

### 2.1) pandas data.frame format
A pandas data.frame with columns `sample`, `group`, `feature`, `view`, `value`.  This is the most intuitive format, as it summarises all omics/groups in a single data structure. Also, there is no need to add rows that correspond to missing data.

For example:
```
sample  group   feature value   view
sample1  groupA    gene1   2.8044  RNA
sample1  groupA    gene3    2.2069  RNA
sample2  groupB    gene2    0.1454  RNA
sample2  groupB    gene1     2.7021  RNA
sample2  groupB    promoter1    3.8618  Methylation
sample3  groupB    promoter2    3.2545  Methylation
sample3  groupB    promoter3   1.5014  Methylation
```

Here we load a simulated data set with the following dimensions:

In [16]:
D = [1000,1000] # Number of features per view
M = len(D)      # Number of views
K = 5           # Number of factors
N = [100,100]   # Number of samples per group
G = len(N)      # Number of groups

data_dt = pd.read_csv("/Users/ricard/data/mofaplus/test/data.txt.gz", sep="\t")
data_dt.head()

,sample,group,feature,view,value
0,sample_0_group_0,group_0,feature_0_view_0,view_0,-2.05
1,sample_1_group_0,group_0,feature_0_view_0,view_0,0.10
2,sample_2_group_0,group_0,feature_0_view_0,view_0,1.44
3,sample_3_group_0,group_0,feature_0_view_0,view_0,-0.28
4,sample_4_group_0,group_0,feature_0_view_0,view_0,-0.88


### 2.2) List of matrices
A nested list of numpy arrays, where the first index refers to the view and the second index refers to the group. Samples are stored in the rows and features are stored in the columns. All views for a given group G must have the same samples in the rows. If there is any sample that is missing a particular view, the column needs to be filled with NAs.

Loading the same data above in matrix format:

In [17]:
data_prefix = "/Users/ricard/data/mofaplus/test"
data_mat = [[None for g in range(G)] for m in range(M)]
for m in range(M):
    for g in range(G):
        data_mat[m][g] = np.loadtxt("%s/%d_%d.txt.gz" % (data_prefix,m,g), delimiter="\t")

### 2.3 Define data options
- **likelihoods**: likelihood per view (options are "gaussian", "poisson", "bernoulli")
- **scale_groups**: if groups have different ranges/variances, it is good practice to scale each group to unit variance. Default is False
- **scale_views**: if views have different ranges/variances, it is good practice to scale each view to unit variance. Default is False

In [18]:
lik = ["gaussian", "gaussian"]

ent.set_data_options(
    likelihoods = lik, 
    scale_groups = False, 
    scale_views = False
)

### 2.4) Add the data to the model

This has to be run after defining the data options

In [19]:
# option 1: data.frame format (slower)
ent.set_data_df(data_dt)

# option 2: nested matrix format (faster)
ent.set_data_matrix(data_mat)

# AnnData format
# (...)

Loaded group='group_0' view='view_0' with N=100 samples and D=1000 features...
Loaded group='group_0' view='view_1' with N=100 samples and D=1000 features...
Loaded group='group_1' view='view_0' with N=100 samples and D=1000 features...
Loaded group='group_1' view='view_1' with N=100 samples and D=1000 features...


View names not provided, using default naming convention:
- view1, view2, ..., viewM
Features names not provided, using default naming convention:
- feature1_view1, featureD_viewM

Groups names not provided, using default naming convention:
- group1, group2, ..., groupG
Samples names not provided, using default naming convention:
- sample1_group1, sample2_group1, sample1_group2, ..., sampleN_groupG

Loaded view='view0' group='group0' with N=100 samples and D=1000 features...
Loaded view='view0' group='group1' with N=100 samples and D=1000 features...
Loaded view='view1' group='group0' with N=100 samples and D=1000 features...
Loaded view='view1' group='group1' with N=100 sa

## 3) Set model options

- **factors**: number of factors
- **likelihods**: same as in data_opts
- **spikeslab_weights**: use spike-slab sparsity prior in the weights? default is TRUE
- **ard_factors**: use ARD prior in the factors? Default is TRUE if using multiple groups.
- **ard_weights**: use ARD prior in the weights? Default is TRUE. 

Only change the default model options if you are familiar with the underlying mathematical model!


In [20]:
ent.set_model_options(
    factors = 10, 
    likelihoods = lik, 
    spikeslab_weights = True, 
    ard_factors = True,
    ard_weights = True
)

## 4) Set training options

- **iter**: number of iterations. Default is 1000.
- **convergence_mode**: "fast", "medium", "slow". For exploration, the fast mode is good enough.
- **startELBO**: initial iteration to compute the ELBO (the objective function used to assess convergence)
- **elbofreq**: frequency of computations of the ELBO (the objective function used to assess convergence)
- **dropR2**: minimum variance explained criteria to drop factors while training
gpu_mode: use GPU mode? (needs cupy installed and a functional GPU, see https://cupy.chainer.org/)
- **verbose**: verbose mode?
- **seed**: random seed

In [21]:
ent.set_train_options(
    iter = 1000, 
    convergence_mode = "fast", 
    startELBO = 1, 
    elbofreq = 1, 
    dropR2 = 0.001, 
    gpu_mode = True, 
    verbose = False, 
    seed = 1
)

GPU mode is activated, but GPU not found... switching to CPU mode
For GPU mode, you need:
1 - Make sure that you are running MOFA+ on a machine with an NVIDIA GPU
2 - Install CUPY following instructions on https://docs-cupy.chainer.org/en/stable/install.html



# 5) (Optional)  stochastic inference options

If the number of samples is very large (at the order of >1e4), you may want to try the stochastic inference scheme. If combined with GPUs, it makes inference significantly faster. However, it requires some additional hyperparameters that in some data sets may need to be optimised (vignette in preparation).

- **batch_size**: float value indicating the batch size (as a fraction of the total data set: 0.10, 0.25 or 0.50)
- **learning_rate**: learning rate (we recommend values from 0.25 to 0.75)
- **forgetting_rate**: forgetting rate (we recommend values from 0.1 to 0.5)


In [22]:
# We do not want to use stochastic inference for this data set

# ent.set_stochastic_options(
# batch_size = 0.5,
# learning_rate = 0.5, 
# forgetting_rate = 0.25
#)

## 6) Build and train the MOFA object 

After training, the model will be saved as an hdf5 file

In [23]:
ent.build()

ent.run()

# Save the output
outfile = "/Users/ricard/data/mofaplus/hdf5/test.hdf5"
ent.save(outfile)



######################################
## Training the model with seed 1 ##
######################################


ELBO before training: -8835540.98


Iteration 1: time=0.06, ELBO=-487709.30, deltaELBO=8347831.676 (94.480142142%), Factors=9
Iteration 2: time=0.06, ELBO=-469049.25, deltaELBO=18660.053 (0.211193100%), Factors=8
Iteration 3: time=0.06, ELBO=-464646.79, deltaELBO=4402.457 (0.049826687%), Factors=7
Iteration 4: time=0.05, ELBO=-461435.87, deltaELBO=3210.924 (0.036340999%), Factors=6
Iteration 5: time=0.05, ELBO=-458961.36, deltaELBO=2474.510 (0.028006323%), Factors=5
Iteration 6: time=0.04, ELBO=-458747.23, deltaELBO=214.129 (0.002423501%), Factors=5
Iteration 7: time=0.04, ELBO=-458622.66, deltaELBO=124.566 (0.001409827%), Factors=5
Iteration 8: time=0.04, ELBO=-458523.29, deltaELBO=99.373 (0.001124700%), Factors=5
Iteration 9: time=0.04, ELBO=-458427.14, deltaELBO=96.153 (0.001088255%), Factors=5
Iteration 10: time=0.04, ELBO=-458325.97, deltaELBO=101.166 (0.001144989

Iteration 96: time=0.06, ELBO=-454370.90, deltaELBO=1.491 (0.000016878%), Factors=5
Iteration 97: time=0.05, ELBO=-454369.38, deltaELBO=1.523 (0.000017236%), Factors=5
Iteration 98: time=0.07, ELBO=-454367.82, deltaELBO=1.562 (0.000017676%), Factors=5
Iteration 99: time=0.04, ELBO=-454366.21, deltaELBO=1.608 (0.000018204%), Factors=5
Iteration 100: time=0.03, ELBO=-451124.79, deltaELBO=3241.416 (0.036686107%), Factors=5
Iteration 101: time=0.04, ELBO=-451078.00, deltaELBO=46.794 (0.000529606%), Factors=5
Iteration 102: time=0.03, ELBO=-451059.04, deltaELBO=18.959 (0.000214575%), Factors=5
Iteration 103: time=0.03, ELBO=-451049.26, deltaELBO=9.786 (0.000110759%), Factors=5
Iteration 104: time=0.03, ELBO=-451044.33, deltaELBO=4.929 (0.000055786%), Factors=5
Iteration 105: time=0.03, ELBO=-451041.68, deltaELBO=2.643 (0.000029918%), Factors=5
Iteration 106: time=0.03, ELBO=-451040.05, deltaELBO=1.628 (0.000018430%), Factors=5
Iteration 107: time=0.03, ELBO=-451038.87, deltaELBO=1.187 (0.00

Iteration 193: time=0.04, ELBO=-450651.40, deltaELBO=31.583 (0.000357454%), Factors=5
Iteration 194: time=0.04, ELBO=-450615.44, deltaELBO=35.962 (0.000407019%), Factors=5
Iteration 195: time=0.05, ELBO=-450574.53, deltaELBO=40.911 (0.000463027%), Factors=5
Iteration 196: time=0.05, ELBO=-450527.92, deltaELBO=46.606 (0.000527489%), Factors=5
Iteration 197: time=0.05, ELBO=-450474.44, deltaELBO=53.483 (0.000605313%), Factors=5
Iteration 198: time=0.04, ELBO=-450412.99, deltaELBO=61.455 (0.000695546%), Factors=5
Iteration 199: time=0.04, ELBO=-450344.72, deltaELBO=68.265 (0.000772622%), Factors=5
Iteration 200: time=0.04, ELBO=-450274.34, deltaELBO=70.380 (0.000796556%), Factors=5
Iteration 201: time=0.04, ELBO=-450209.18, deltaELBO=65.163 (0.000737510%), Factors=5
Iteration 202: time=0.05, ELBO=-450155.72, deltaELBO=53.460 (0.000605061%), Factors=5
Iteration 203: time=0.05, ELBO=-450114.53, deltaELBO=41.188 (0.000466165%), Factors=5
Iteration 204: time=0.04, ELBO=-450080.19, deltaELBO=3

## 7) Downstream analysis

This finishes the tutorial on how to train a MOFA object from python. To continue with the downstream analysis you have to switch to R. Please, follow [this tutorial](https://raw.githack.com/bioFAM/MOFA2/master/MOFA2/vignettes/downstream_analysis.html)